## 数据探索与可视化

### 实验目标
- 学习并掌握如何使用Python进行数据探索与可视化。
- 理解并应用数据探索的基本方法，包括数据清洗、格式化和描述性统计分析。
- 使用Matplotlib和Seaborn等可视化工具，绘制不同类型的图表，分析数据分布及特征之间的关系。
- 利用pandas_profiling生成数据的交互式报告，全面了解数据的分布、缺失值和异常值。

### 实验要求

1. 使用pandas读取数据集github_bot_processed_data.csv。探索pandas的显示选项，以便查看更多数据行和列。
2. 使用head()方法查看数据的前几行，并使用info()方法查看每列的数据类型，观察两者的区别。
使用describe()方法生成数据的描述性统计信息。
3. 对数据进行格式化处理，并展示不同格式（例如，日期、货币、百分比等）的效果。
4. 对public_repos、public_gists、followers、following等列进行对数变换，并查看其影响。
5. 使用Matplotlib绘制图表
    - 绘制条形图：展示label列的类别分布。
    - 绘制堆积柱状图：展示多个布尔特征（如site_admin、company等）的分布。
    - 绘制直方图：展示log_public_repos的对数变换后的数据分布。
    - 绘制散点图：展示public_repos与followers之间的关系。
    - 绘制散点矩阵：展示多个数值型特征之间的成对关系。
6. 使用Seaborn绘制图表
    - 绘制箱线图：展示不同label类别下log_followers的分布。
    - 绘制成对图：展示不同特征之间的成对关系，并根据label分类。
    - 绘制热图：展示log_public_repos、log_public_gists、log_followers和log_following等特征之间的相关性。
    - 绘制小提琴图：展示label与log_followers之间的分布差异。
7. 使用pandas_profiling.ProfileReport()生成交互式数据分析报告，分析数据的统计分布、缺失值、异常值等。

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling

# 读取数据集
df = pd.read_csv('github_bot_processed_data.csv')

# 设置pandas显示选项，以便查看更多数据行和列
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 使用head()方法查看数据的前几行
print("Data head:")
print(df.head())

# 使用info()方法查看每列的数据类型
print("\nData info:")
print(df.info())

# 使用describe()方法生成数据的描述性统计信息
print("\nData description:")
print(df.describe())

# 对数据进行格式化处理，并展示不同格式（例如，日期、货币、百分比等）的效果
# 假设created_at是日期列，followers是数值列：
df['created_at'] = pd.to_datetime(df['created_at'])
df['followers'] = df['followers'].apply(lambda x: "${:,.2f}".format(x))

print("\nFormatted data:")
print(df.head())

# 对public_repos、public_gists、followers、following等列进行对数变换，并查看其影响
log_columns = ['public_repos', 'public_gists', 'followers', 'following']
for col in log_columns:
    df[col + '_log'] = df[col].apply(lambda x: np.log1p(x) if x > 0 else 0)

print("\nLog transformed columns:")
print(df[log_columns + [col + '_log' for col in log_columns]].head())

# 使用Matplotlib绘制图表
# 条形图：展示label列的类别分布。
plt.figure(figsize=(10, 6))
df['label'].value_counts().plot(kind='bar')
plt.title('Label Distribution')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

# 堆积柱状图：展示多个布尔特征（如site_admin、company等）的分布。
plt.figure(figsize=(10, 6))
df[['site_admin', 'company']].sum().plot(kind='bar', stacked=True)
plt.title('Boolean Features Distribution')
plt.xlabel('Feature')
plt.ylabel('Count')
plt.show()

# 直方图：展示log_public_repos的对数变换后的数据分布。
plt.figure(figsize=(10, 6))
sns.histplot(df['public_repos_log'], bins=30, kde=True)
plt.title('Log Public Repos Distribution')
plt.xlabel('Log Public Repos')
plt.ylabel('Frequency')
plt.show()

# 散点图：展示public_repos与followers之间的关系。
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='public_repos', y='followers')
plt.title('Public Repos vs Followers')
plt.xlabel('Public Repos')
plt.ylabel('Followers')
plt.show()

# 散点矩阵：展示多个数值型特征之间的成对关系。
pd.plotting.scatter_matrix(df[['public_repos', 'public_gists', 'followers', 'following']], figsize=(12, 8))
plt.suptitle('Scatter Matrix of Numeric Features')
plt.show()

# 使用Seaborn绘制图表
# 箱线图：展示不同label类别下log_followers的分布。
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='label', y='followers_log')
plt.title('Boxplot of Log Followers by Label')
plt.xlabel('Label')
plt.ylabel('Log Followers')
plt.show()

# 成对图：展示不同特征之间的成对关系，并根据label分类。
sns.pairplot(df, hue='label', markers=["o", "s", "D"])
plt.suptitle('Pairplot with Hue by Label')
plt.show()

# 热图：展示log_public_repos、log_public_gists、log_followers和log_following等特征之间的相关性。
correlation_matrix = df[[col + '_log' for col in log_columns]].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Heatmap')
plt.show()

# 小提琴图：展示label与log_followers之间的分布差异。
sns.violinplot(data=df, x='label', y='followers_log')
plt.title('Violin Plot of Log Followers by Label')
plt.xlabel('Label')
plt.ylabel('Log Followers')
plt.show()

# 使用pandas_profiling生成交互式数据分析报告
profile = pandas_profiling.ProfileReport(df, title="GitHub Data Report", explorative=True)
profile.to_file("github_data_report.html")


PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.9/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.9/u/import-error